In [ ]:
#IMPORT PYTHON PACKAGES
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.integrate
from scipy.stats import sem
import numpy  as np
import copy
import matplotlib.pylab as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import time
import datetime
import subprocess
import itertools
from itertools import groupby, repeat, islice
import math as math
import decimal as d
from decimal import *

#%matplotlib notebook
#inline
#Plot settings
sns.set_style("whitegrid", rc={"axes.edgecolor": "k", "axes.linewidth":2.})

sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})


sns.set_context("notebook",rc={"grid.linewidth": 0, 
                            "font.family":"Helvetica", "axes.labelsize":24.,"xtick.labelsize":24., 
                            "ytick.labelsize":24., "legend.fontsize":18.})

color_blind_safe = sns.color_palette("colorblind", 10) 

colors = sns.color_palette("tab10", 10) 


In [ ]:
#400

filename = ['./profile_siE400r50','profile_siE400r125', 'profile_siE400r250', 'profile_siE400r271']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity


Thermo_Data = np.array([np.loadtxt('./Thermo_siE400r50'), np.loadtxt('Thermo_siE400r125'), np.loadtxt('Thermo_siE400r250'), np.loadtxt('Thermo_siE400r271')], dtype=object) #, np.loadtxt('Thermo_siE400r125_Original_2')])
                                                                      

labels = ['SiGeSi400r50','SiGeSi400r125', 'SiGeSi400r250', 'SiGeSi400r271'] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]
    
# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                        ], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                      ], dtype=float)

excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];temp_D_400Si = [];mid_temp_400Si = []; average_temp_400Si = [];
boundary_low_400Si = []; boundary_high_400Si = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
   
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    
    xhigh_left = dX[230:360]
    yhigh_left = dTemp[230:360]
   
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c
    
    xlow_left = dX[10:130]
    ylow_left = dTemp[10:130]
 
    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept

    y_high_4 = (high_slope * dX[200]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/4.05)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_400Si.append(temp_diff_4)
    
    x_axis_temp_loc = (1/4.05)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    
    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_400Si.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_400Si.append(average_temp)

    boundary_high = dTemp[excluded_points1]
    boundary_high_400Si.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_400Si.append(boundary_low)

    
Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_400 = []
ThermalsGe125 = []
hf_values = [];heat_fluxSi400Si_values = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.15f' % elem for elem in Therm_val]
    ThermalsGe125.append(Ttherm)    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.15f' % elem for elem in TC_values]
    s_values.append(val)    
    
    LR400 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    value = ['%.15f' % elem for elem in LR400]
    LR_values_400.append(value)
    
    
    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.15f' % elem for elem in fluxes]
    
    heat_fluxSi400Si_values.append(heat_val)    
   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_400Si = []
    SUM_HF_400Si.append(sum_hf_values)
    
    
heat_flux_top_left400Si = [];heat_flux_top_right400Si = [];heat_flux_bot_left400Si = [];heat_flux_bot_right400Si = [];
for h in hf_values:
    heat_flux_top_left400Si.append(np.array(float(h[0]))) #flux_top_left
    #heat_flux_top_right400Si.append(np.array(float(h[1]))) flux_top_right
    #heat_flux_bot_left400Si.append(np.array(float(h[2]))) flux_bot_left
    #heat_flux_bot_right400Si.append(np.array(float(h[3]))) flux_bot_right


therm_averageSi400 = []
for item in s_values:
    therm_averageSi400.append(item[2])

therm_averageSi400 = []
for item in s_values:
    therm_averageSi400.append(np.array(float(item[2])))


av_silicon_SiGeSi_400 = (float(LR_values_400[0][0]) + float(LR_values_400[0][3]) +
               float(LR_values_400[1][0]) + float(LR_values_400[1][3]) +
               float(LR_values_400[2][0]) + float(LR_values_400[2][3]) +
               float(LR_values_400[3][0]) + float(LR_values_400[3][3])) / 8

av_silicon_SiGeSi_400_values = [float(LR_values_400[0][0]), float(LR_values_400[0][3]),
               float(LR_values_400[1][0]), float(LR_values_400[1][3]),
               float(LR_values_400[2][0]), float(LR_values_400[2][3]),
               float(LR_values_400[3][0]), float(LR_values_400[3][3])]



av_germanium_SiGeSi_400 = (float(LR_values_400[0][1]) + float(LR_values_400[0][2]) +
               float(LR_values_400[1][1]) + float(LR_values_400[1][2]) +
               float(LR_values_400[2][1]) + float(LR_values_400[2][2]) +
               float(LR_values_400[3][1]) + float(LR_values_400[3][2])) / 8

av_germanium_SiGeSi_400_values = [float(LR_values_400[0][1]), float(LR_values_400[0][2]),
               float(LR_values_400[1][1]), float(LR_values_400[1][2]),
               float(LR_values_400[2][1]), float(LR_values_400[2][2]),
               float(LR_values_400[3][1]), float(LR_values_400[3][2])]

getcontext().prec = 10
std_silicon_SiGeSi_400 = Decimal(np.std([float(LR_values_400[0][0]),float(LR_values_400[0][3]),
               float(LR_values_400[1][0]),float(LR_values_400[1][3]),
               float(LR_values_400[2][0]),float(LR_values_400[2][3]),
               float(LR_values_400[3][0]),float(LR_values_400[3][3])], dtype=np.float64))
                      
SE_silicon_SiGeSi_400 = std_silicon_SiGeSi_400/Decimal(math.sqrt(8))

getcontext().prec = 10
std_germanium_SiGeSi_400 = Decimal(np.std([float(LR_values_400[0][1]),float(LR_values_400[0][2]),
               float(LR_values_400[1][1]),float(LR_values_400[1][2]),
               float(LR_values_400[2][1]),float(LR_values_400[2][2]),
               float(LR_values_400[3][1]),float(LR_values_400[3][2])], dtype=np.float64))
                      
SE_germanium_SiGeSi_400 = std_germanium_SiGeSi_400/Decimal(math.sqrt(8))

sil = np.array([float(LR_values_400[0][0]),float(LR_values_400[0][3]),
               float(LR_values_400[1][0]),float(LR_values_400[1][3]),
               float(LR_values_400[2][0]),float(LR_values_400[2][3]),
               float(LR_values_400[3][0]),float(LR_values_400[3][3])])
ger = np.array([float(LR_values_400[0][1]),float(LR_values_400[0][2]),
               float(LR_values_400[1][1]),float(LR_values_400[1][2]),
               float(LR_values_400[2][1]),float(LR_values_400[2][2]),
               float(LR_values_400[3][1]),float(LR_values_400[3][2])])

sil_SiGeSi_400_50 = np.array([sil[0],sil[1]])
sil_SiGeSi_400_125 = np.array([sil[2],sil[3]])
sil_SiGeSi_400_250 = np.array([sil[4],sil[5]])
sil_SiGeSi_400_271 = np.array([sil[6],sil[7]])

ger_SiGeSi_400_50 = np.array([ger[0],ger[1]])
ger_SiGeSi_400_125 = np.array([ger[2],ger[3]])
ger_SiGeSi_400_250 = np.array([ger[4],ger[5]])
ger_SiGeSi_400_271 = np.array([ger[6],ger[7]])

In [ ]:
#200
filename = ['./profile_geE200r50','profile_geE200r125', 'profile_geE200r250', 'profile_geE200r271']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_geE200r50'), np.loadtxt('Thermo_geE200r125'), np.loadtxt('Thermo_geE200r250'), np.loadtxt('Thermo_geE200r271')], dtype=object) #, np.loadtxt('Thermo_geE200r125_Original_2')])
                                                                      

labels = ['GeSiGe200r50','GeSiGe200r125', 'GeSiGe200r250', 'GeSiGe200r271'] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                        ], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                      ], dtype=float)


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 120
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];temp_D_200Ge = [];mid_temp_200Ge = []; average_temp_200Ge = [];
boundary_low_200Ge = []; boundary_high_200Ge = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])

    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
   
    xhigh_left = dX[230:390]
    yhigh_left = dTemp[230:390]
   
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c
    
    xlow_left = dX[10:170]
    ylow_left = dTemp[10:170]

    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept
      
    y_high_4 = (high_slope * dX[205]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/3.9)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_200Ge.append(temp_diff_4)

    x_axis_temp_loc = (1/3.9)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_200Ge.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_200Ge.append(average_temp)

    boundary_high = dTemp[excluded_points1]
    boundary_high_200Ge.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_200Ge.append(boundary_low)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_200 = []
ThermalsGe125 = []
hf_values = [];heat_flux200Ge_values = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.15f' % elem for elem in Therm_val]
    ThermalsGe125.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.15f' % elem for elem in TC_values]
    s_values.append(val)
    
    LR200 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    value = ['%.15f' % elem for elem in LR200]
    LR_values_200.append(value)

    df = Slope2[simNo,0]*1e10    

    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.15f' % elem for elem in fluxes]
    
    heat_flux200Ge_values.append(heat_val)
   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_200Ge = []
    SUM_HF_200Ge.append(sum_hf_values)

    
    
heat_flux_top_left200Ge = [];heat_flux_top_right200Ge = [];heat_flux_bot_left200Ge = [];heat_flux_bot_right200Ge = [];
for h in hf_values:
    heat_flux_top_left200Ge.append(np.array(float(h[0]))) #flux_top_left
    #heat_flux_top_right200Ge.append(np.array(float(h[1]))) flux_top_right
    #heat_flux_bot_left200Ge.append(np.array(float(h[2]))) flux_bot_left
    #heat_flux_bot_right200Ge.append(np.array(float(h[3]))) flux_bot_right





therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(item[2])

therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(np.array(float(item[2])))


av_germanium_GeSiGe_200 = (float(LR_values_200[0][0]) + float(LR_values_200[0][3]) +
               float(LR_values_200[1][0]) + float(LR_values_200[1][3]) +
               float(LR_values_200[2][0]) + float(LR_values_200[2][3]) +
               float(LR_values_200[3][0]) + float(LR_values_200[3][3])) / 8

av_germanium_GeSiGe_200_values = [float(LR_values_200[0][0]), float(LR_values_200[0][3]),
               float(LR_values_200[1][0]), float(LR_values_200[1][3]),
               float(LR_values_200[2][0]), float(LR_values_200[2][3]),
               float(LR_values_200[3][0]), float(LR_values_200[3][3])]


av_silicon_GeSiGe_200 = (float(LR_values_200[0][1]) + float(LR_values_200[0][2]) +
               float(LR_values_200[1][1]) + float(LR_values_200[1][2]) +
               float(LR_values_200[2][1]) + float(LR_values_200[2][2]) +
               float(LR_values_200[3][1]) + float(LR_values_200[3][2])) / 8

av_silicon_GeSiGe_200_values = [float(LR_values_200[0][1]), float(LR_values_200[0][2]),
               float(LR_values_200[1][1]), float(LR_values_200[1][2]),
               float(LR_values_200[2][1]), float(LR_values_200[2][2]),
               float(LR_values_200[3][1]), float(LR_values_200[3][2])]

getcontext().prec = 10
std_germanium_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])], dtype=np.float64))
                      
SE_germanium_GeSiGe_200 = std_germanium_GeSiGe_200/Decimal(math.sqrt(8))

getcontext().prec = 10
std_silicon_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])], dtype=np.float64))
                      
SE_silicon_GeSiGe_200 = std_silicon_GeSiGe_200/Decimal(math.sqrt(8))

ger = np.array([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])])
sil = np.array([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])])

ger_GeSiGe_200_50 = np.array([ger[0],ger[1]])
ger_GeSiGe_200_125 = np.array([ger[2],ger[3]])
ger_GeSiGe_200_250 = np.array([ger[4],ger[5]])
ger_GeSiGe_200_271 = np.array([ger[6],ger[7]])

sil_GeSiGe_200_50 = np.array([sil[0],sil[1]])
sil_GeSiGe_200_125 = np.array([sil[2],sil[3]])
sil_GeSiGe_200_250 = np.array([sil[4],sil[5]])
sil_GeSiGe_200_271 = np.array([sil[6],sil[7]])

In [ ]:
#R_int = delta T/J
#J = delta KE/Ac.delta t
#delta t = 0.5 fs
A_t_nm = 0.1
A_t_m = 1e-10


radius_sq = ((54.3*A_t_m)/2)**2
print(((54.3*A_t_m)/2))
print(54.3*A_t_m)

cross_section = np.pi*radius_sq

d_t = 3e-9

Ac_dt = 2* d_t

In [ ]:
#Si
#200
R_int_s200r50 = temp_D_200Si[0]/np.sum((float(heat_fluxSi200Si_values[0][0]),float(heat_fluxSi200Si_values[0][2])), dtype=np.float32)

R_int_s200r125 = temp_D_200Si[1]/np.sum((float(heat_fluxSi200Si_values[1][0]),float(heat_fluxSi200Si_values[1][2])), dtype=np.float32)

R_int_s200r250 = temp_D_200Si[2]/np.sum((float(heat_fluxSi200Si_values[2][0]),float(heat_fluxSi200Si_values[2][2])), dtype=np.float32)

R_int_s200r271 = temp_D_200Si[3]/np.sum((float(heat_fluxSi200Si_values[3][0]),float(heat_fluxSi200Si_values[3][2])), dtype=np.float32)


In [ ]:
#Ge
#200
#print(heat_flux200Ge_values[0][2])
#print(temp_D_200Ge[0])
R_int_g200r50 = temp_D_200Ge[0]/np.sum((float(heat_flux200Ge_values[0][0]),float(heat_flux200Ge_values[0][2])), dtype=np.float32)

R_int_g200r125 = temp_D_200Ge[1]/np.sum((float(heat_flux200Ge_values[1][0]),float(heat_flux200Ge_values[1][2])), dtype=np.float32)

R_int_g200r250 = temp_D_200Ge[2]/np.sum((float(heat_flux200Ge_values[2][0]),float(heat_flux200Ge_values[2][2])), dtype=np.float32)

R_int_g200r271 = temp_D_200Ge[3]/np.sum((float(heat_flux200Ge_values[3][0]),float(heat_flux200Ge_values[3][2])), dtype=np.float32)



In [ ]:
#SIGESI
Int_R_SI200 = [R_int_s200r50,R_int_s200r125,R_int_s200r250,R_int_s200r271]
Int_R_SI400 = [R_int_s400r50,R_int_s400r125,R_int_s400r250,R_int_s400r271]
Int_R_SI600 = [R_int_s600r50,R_int_s600r125,R_int_s600r250,R_int_s600r271]
Int_R_SI800 = [R_int_s800r50,R_int_s800r125,R_int_s800r250,R_int_s800r271]
Int_R_SI1000 = [R_int_s1000r50,R_int_s1000r125,R_int_s1000r250,R_int_s1000r271]

INT_R_SI_ALL = [Int_R_SI200,Int_R_SI400,Int_R_SI600,Int_R_SI800,Int_R_SI1000]#,Int_R_SI1500,Int_R_SI2k]
MID_TEMP_SI_ALL = [mid_temp_200Si,mid_temp_400Si,mid_temp_600Si,mid_temp_800Si,mid_temp_1000Si]#,mid_temp_1500Si,mid_temp_2kSi]
TEMP_SI_ALL = [temp_D_200Si,temp_D_400Si,temp_D_600Si,temp_D_800Si,temp_D_1000Si]
BOUNDARY_LOW_SI_ALL = [boundary_low_200Si,boundary_low_400Si,boundary_low_600Si,boundary_low_800Si,boundary_low_1000Si]
BOUNDARY_HIGH_SI_ALL = [boundary_high_200Si,boundary_high_400Si,boundary_high_600Si,boundary_high_800Si,boundary_high_1000Si]


#GESIGE
Int_R_GE200 = [R_int_g200r50,R_int_g200r125,R_int_g200r250,R_int_g200r271]
Int_R_GE400 = [R_int_g400r50,R_int_g400r125,R_int_g400r250,R_int_g400r271]
Int_R_GE600 = [R_int_g600r50,R_int_g600r125,R_int_g600r250,R_int_g600r271]
Int_R_GE800 = [R_int_g800r50,R_int_g800r125,R_int_g800r250,R_int_g800r271]
Int_R_GE1000 = [R_int_g1000r50,R_int_g1000r125,R_int_g1000r250,R_int_g1000r271]

INT_R_GE_ALL = [Int_R_GE200,Int_R_GE400,Int_R_GE600,Int_R_GE800,Int_R_GE1000]#,Int_R_GE1500,Int_R_GE2k]
MID_TEMP_GE_ALL = [mid_temp_200Ge,mid_temp_400Ge,mid_temp_600Ge,mid_temp_800Ge,mid_temp_1000Ge]#,mid_temp_1500Ge,mid_temp_2kGe]
TEMP_GE_ALL = [temp_D_200Ge,temp_D_400Ge,temp_D_600Ge,temp_D_800Ge,temp_D_1000Ge]
BOUNDARY_LOW_GE_ALL = [boundary_low_200Ge,boundary_low_400Ge,boundary_low_600Ge,boundary_low_800Ge,boundary_low_1000Ge]
BOUNDARY_HIGH_GE_ALL = [boundary_high_200Ge,boundary_high_400Ge,boundary_high_600Ge,boundary_high_800Ge,boundary_high_1000Ge]



In [ ]:
rad_size1 = np.ones(4)*110.0
rad_size2 = np.ones(4)*219.9
rad_size3 = np.ones(4)*329.9
rad_size4 = np.ones(4)*439.9
rad_size5 = np.ones(4)*550.0
rad_size = [rad_size1,rad_size2,rad_size3,rad_size4,rad_size5]

rad_sizeG1 = np.ones(4)*110.0
rad_sizeG2 = np.ones(4)*219.9
rad_sizeG3 = np.ones(4)*329.9
rad_sizeG4 = np.ones(4)*439.9
rad_sizeG5 = np.ones(4)*550.0
rad_sizeG = [rad_sizeG1,rad_sizeG2,rad_sizeG3,rad_sizeG4,rad_sizeG5]


In [ ]:
#size
all_boxes_si_size = []
for t in rad_size:
    for i in t:
        all_boxes_si_size.append(i)
print(np.shape(all_boxes_si_size))
print(all_boxes_si_size)

all_boxes_ge_size = []
for t in rad_sizeG:
    for i in t:
        all_boxes_ge_size.append(i)

#IntR
all_boxes_si_IntR = []
for t in INT_R_SI_ALL:
    for i in t:
        all_boxes_si_IntR.append(i)
all_boxes_si_IntR = [i/1e8 for i in all_boxes_si_IntR]
print(np.shape(all_boxes_si_IntR))

all_boxes_ge_IntR = []
for t in INT_R_GE_ALL:
    for i in t:
        all_boxes_ge_IntR.append(i)
all_boxes_ge_IntR = [i/1e8 for i in all_boxes_ge_IntR]

#midT
all_boxes_si_midT = []
for t in MID_TEMP_SI_ALL:
    for i in t:
        all_boxes_si_midT.append(i)

all_boxes_ge_midT = []
for t in MID_TEMP_GE_ALL:
    for i in t:
        all_boxes_ge_midT.append(i)

#TempD
all_boxes_si_TempD = []
for t in TEMP_SI_ALL:
    for i in t:
        all_boxes_si_TempD.append(i)

all_boxes_ge_TempD = []
for t in TEMP_GE_ALL:
    for i in t:
        all_boxes_ge_TempD.append(i)

#upperBoundary
all_boxes_si_upperBoundary = []
for t in BOUNDARY_HIGH_SI_ALL:
    for i in t:
        all_boxes_si_upperBoundary.append(i)

all_boxes_ge_upperBoundary = []
for t in BOUNDARY_HIGH_GE_ALL:
    for i in t:
        all_boxes_ge_upperBoundary.append(i)

#lowerBoundary
all_boxes_si_lowerBoundary = []
for t in BOUNDARY_LOW_SI_ALL:
    for i in t:
        all_boxes_si_lowerBoundary.append(i)

all_boxes_ge_lowerBoundary = []
for t in BOUNDARY_LOW_GE_ALL:
    for i in t:
        all_boxes_ge_lowerBoundary.append(i)


In [ ]:
#2D plots

In [ ]:
#size vs Int_R

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

xline_si = rad_size
zline_si = INT_R_SI_ALL
yline_si = MID_TEMP_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
zline = INT_R_GE_ALL
yline = MID_TEMP_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

#Si
#200
plt.scatter(xline_si[0][0], zline_si[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(xline_si[0][1], zline_si[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(xline_si[0][2], zline_si[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(xline_si[0][3], zline_si[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

#400
plt.scatter(xline_si[1][0], zline_si[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[1][1], zline_si[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[1][2], zline_si[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[1][3], zline_si[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(xline_si[2][0], zline_si[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[2][1], zline_si[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[2][2], zline_si[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[2][3], zline_si[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(xline_si[3][0], zline_si[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[3][1], zline_si[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[3][2], zline_si[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[3][3], zline_si[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(xline_si[4][0], zline_si[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[4][1], zline_si[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[4][2], zline_si[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[4][3], zline_si[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(xline[0][0], zline[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(xline[0][1], zline[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(xline[0][2], zline[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(xline[0][3], zline[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(xline[1][0], zline[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[1][1], zline[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[1][2], zline[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[1][3], zline[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#600
plt.scatter(xline[2][0], zline[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[2][1], zline[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[2][2], zline[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[2][3], zline[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(xline[3][0], zline[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[3][1], zline[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[3][2], zline[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[3][3], zline[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#1000
plt.scatter(xline[4][0], zline[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[4][1], zline[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[4][2], zline[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[4][3], zline[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(329.0, 1.75,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(329.0, 0.45,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(329.0, 0.6), xytext=(329.0, 0.6), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(329.0, 1.9), xytext=(329.0, 1.9), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 


plt.show()  


In [ ]:
#Size vs Mid.temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


xline_si = rad_size
yline_si = MID_TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = MID_TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

#Si
#200
plt.scatter(xline_si[0][0], yline_si[0][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(xline_si[0][1], yline_si[0][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(xline_si[0][2], yline_si[0][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(xline_si[0][3], yline_si[0][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

print(xline_si[0][0])
#400
plt.scatter(xline_si[1][0], yline_si[1][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[1][1], yline_si[1][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[1][2], yline_si[1][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[1][3], yline_si[1][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(xline_si[2][0], yline_si[2][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[2][1], yline_si[2][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[2][2], yline_si[2][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[2][3], yline_si[2][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(xline_si[3][0], yline_si[3][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[3][1], yline_si[3][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[3][2], yline_si[3][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[3][3], yline_si[3][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(xline_si[4][0], yline_si[4][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[4][1], yline_si[4][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[4][2], yline_si[4][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[4][3], yline_si[4][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(xline[0][0], yline[0][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(xline[0][1], yline[0][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(xline[0][2], yline[0][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(xline[0][3], yline[0][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(xline[1][0], yline[1][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[1][1], yline[1][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[1][2], yline[1][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[1][3], yline[1][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#600
plt.scatter(xline[2][0], yline[2][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[2][1], yline[2][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[2][2], yline[2][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[2][3], yline[2][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(xline[3][0], yline[3][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[3][1], yline[3][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[3][2], yline[3][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[3][3], yline[3][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#1000
plt.scatter(xline[4][0], yline[4][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[4][1], yline[4][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[4][2], yline[4][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[4][3], yline[4][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_midT, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(110, 319,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(329, 356,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_midT)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_midT)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(110, 315), xytext=(110, 315), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(329, 360), xytext=(329, 360), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n Middle Temperature (K)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 

plt.show()  

In [ ]:
#Mid.temp vs Int_R

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

xline_si = rad_size
yline_si = MID_TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = MID_TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

#Si
#200
plt.scatter(yline_si[0][0], zline_si[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(yline_si[0][1], zline_si[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(yline_si[0][2], zline_si[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(yline_si[0][3], zline_si[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

#400
plt.scatter(yline_si[1][0], zline_si[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(yline_si[1][1], zline_si[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(yline_si[1][2], zline_si[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(yline_si[1][3], zline_si[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(yline_si[2][0], zline_si[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(yline_si[2][1], zline_si[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(yline_si[2][2], zline_si[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(yline_si[2][3], zline_si[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(yline_si[3][0], zline_si[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(yline_si[3][1], zline_si[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(yline_si[3][2], zline_si[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(yline_si[3][3], zline_si[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(yline_si[2][0], zline_si[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(yline_si[2][1], zline_si[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(yline_si[2][2], zline_si[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(yline_si[2][3], zline_si[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(yline[0][0], zline[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(yline[0][1], zline[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(yline[0][2], zline[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(yline[0][3], zline[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(yline[1][0], zline[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(yline[1][1], zline[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(yline[1][2], zline[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(yline[1][3], zline[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#600
plt.scatter(yline[2][0], zline[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(yline[2][1], zline[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(yline[2][2], zline[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(yline[2][3], zline[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(yline[3][0], zline[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(yline[3][1], zline[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(yline[3][2], zline[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(yline[3][3], zline[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#1000
plt.scatter(yline[4][0], zline[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(yline[4][1], zline[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(yline[4][2], zline[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(yline[4][3], zline[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_midT)
mxG = max(all_boxes_ge_midT)

mnS = min(all_boxes_si_midT)
mxS = max(all_boxes_si_midT)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(320, 2.0,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(350, 1.8,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_midT, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_midT, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(320, 1.8), xytext=(320, 1.8), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(350, 2.0), xytext=(350, 2.0), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Middle Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Rint vs upper boundary temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


#Si
#200
plt.scatter(BOUNDARY_HIGH_SI_ALL[0][0], INT_R_SI_ALL[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(BOUNDARY_HIGH_SI_ALL[0][1], INT_R_SI_ALL[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(BOUNDARY_HIGH_SI_ALL[0][2], INT_R_SI_ALL[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(BOUNDARY_HIGH_SI_ALL[0][3], INT_R_SI_ALL[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

#400
plt.scatter(BOUNDARY_HIGH_SI_ALL[1][0], INT_R_SI_ALL[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[1][1], INT_R_SI_ALL[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[1][2], INT_R_SI_ALL[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[1][3], INT_R_SI_ALL[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(BOUNDARY_HIGH_SI_ALL[2][0], INT_R_SI_ALL[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[2][1], INT_R_SI_ALL[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[2][2], INT_R_SI_ALL[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[2][3], INT_R_SI_ALL[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(BOUNDARY_HIGH_SI_ALL[3][0], INT_R_SI_ALL[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[3][1], INT_R_SI_ALL[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[3][2], INT_R_SI_ALL[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[3][3], INT_R_SI_ALL[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(BOUNDARY_HIGH_SI_ALL[4][0], INT_R_SI_ALL[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[4][1], INT_R_SI_ALL[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[4][2], INT_R_SI_ALL[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_HIGH_SI_ALL[4][3], INT_R_SI_ALL[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(BOUNDARY_HIGH_GE_ALL[0][0], INT_R_GE_ALL[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(BOUNDARY_HIGH_GE_ALL[0][1], INT_R_GE_ALL[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(BOUNDARY_HIGH_GE_ALL[0][2], INT_R_GE_ALL[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(BOUNDARY_HIGH_GE_ALL[0][3], INT_R_GE_ALL[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(BOUNDARY_HIGH_GE_ALL[1][0], INT_R_GE_ALL[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[1][1], INT_R_GE_ALL[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[1][2], INT_R_GE_ALL[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[1][3], INT_R_GE_ALL[1][3]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
#600
plt.scatter(BOUNDARY_HIGH_GE_ALL[2][0], INT_R_GE_ALL[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[2][1], INT_R_GE_ALL[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[2][2], INT_R_GE_ALL[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[2][3], INT_R_GE_ALL[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(BOUNDARY_HIGH_GE_ALL[3][0], INT_R_GE_ALL[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[3][1], INT_R_GE_ALL[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[3][2], INT_R_GE_ALL[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[3][3], INT_R_GE_ALL[3][3]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
#1000
plt.scatter(BOUNDARY_HIGH_GE_ALL[4][0], INT_R_GE_ALL[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[4][1], INT_R_GE_ALL[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[4][2], INT_R_GE_ALL[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_HIGH_GE_ALL[4][3], INT_R_GE_ALL[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_upperBoundary)
mxG = max(all_boxes_ge_upperBoundary)

mnS = min(all_boxes_si_upperBoundary)
mxS = max(all_boxes_si_upperBoundary)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(390, 0.8,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(390, 1.8,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_upperBoundary, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_upperBoundary, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(390, 0.5), xytext=(390, 0.5), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(390, 2.0), xytext=(390, 2.0), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Top Boundary Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Rint vs lower boundary temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

#Si
#200
plt.scatter(BOUNDARY_LOW_SI_ALL[0][0], INT_R_SI_ALL[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(BOUNDARY_LOW_SI_ALL[0][1], INT_R_SI_ALL[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(BOUNDARY_LOW_SI_ALL[0][2], INT_R_SI_ALL[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(BOUNDARY_LOW_SI_ALL[0][3], INT_R_SI_ALL[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

#400
plt.scatter(BOUNDARY_LOW_SI_ALL[1][0], INT_R_SI_ALL[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[1][1], INT_R_SI_ALL[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[1][2], INT_R_SI_ALL[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[1][3], INT_R_SI_ALL[1][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(BOUNDARY_LOW_SI_ALL[2][0], INT_R_SI_ALL[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[2][1], INT_R_SI_ALL[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[2][2], INT_R_SI_ALL[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[2][3], INT_R_SI_ALL[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(BOUNDARY_LOW_SI_ALL[3][0], INT_R_SI_ALL[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[3][1], INT_R_SI_ALL[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[3][2], INT_R_SI_ALL[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[3][3], INT_R_SI_ALL[3][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(BOUNDARY_LOW_SI_ALL[4][0], INT_R_SI_ALL[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[4][1], INT_R_SI_ALL[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[4][2], INT_R_SI_ALL[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(BOUNDARY_LOW_SI_ALL[4][3], INT_R_SI_ALL[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(BOUNDARY_LOW_GE_ALL[0][0], INT_R_GE_ALL[0][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(BOUNDARY_LOW_GE_ALL[0][1], INT_R_GE_ALL[0][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(BOUNDARY_LOW_GE_ALL[0][2], INT_R_GE_ALL[0][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(BOUNDARY_LOW_GE_ALL[0][3], INT_R_GE_ALL[0][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(BOUNDARY_LOW_GE_ALL[1][0], INT_R_GE_ALL[1][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[1][1], INT_R_GE_ALL[1][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[1][2], INT_R_GE_ALL[1][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[1][3], INT_R_GE_ALL[1][3]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
#600
plt.scatter(BOUNDARY_LOW_GE_ALL[2][0], INT_R_GE_ALL[2][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[2][1], INT_R_GE_ALL[2][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[2][2], INT_R_GE_ALL[2][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[2][3], INT_R_GE_ALL[2][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(BOUNDARY_LOW_GE_ALL[3][0], INT_R_GE_ALL[3][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[3][1], INT_R_GE_ALL[3][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[3][2], INT_R_GE_ALL[3][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[3][3], INT_R_GE_ALL[3][3]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
#1000
plt.scatter(BOUNDARY_LOW_GE_ALL[4][0], INT_R_GE_ALL[4][0]/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[4][1], INT_R_GE_ALL[4][1]/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[4][2], INT_R_GE_ALL[4][2]/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(BOUNDARY_LOW_GE_ALL[4][3], INT_R_GE_ALL[4][3]/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)


#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_lowerBoundary)
mxG = max(all_boxes_ge_lowerBoundary)

mnS = min(all_boxes_si_lowerBoundary)
mxS = max(all_boxes_si_lowerBoundary)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(320, 1.8,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(353, 1.8,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_lowerBoundary, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(320, 2.0), xytext=(320, 2.0), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(355, 2.0), xytext=(355, 2.0), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Bottom Boundary Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Size vs TempD

In [ ]:
##### from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


xline_si = rad_size
yline_si = TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

#Si
#200
plt.scatter(xline_si[0][0], yline_si[0][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-r50')
plt.scatter(xline_si[0][1], yline_si[0][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-r125')
plt.scatter(xline_si[0][2], yline_si[0][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-r250')
plt.scatter(xline_si[0][3], yline_si[0][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-r271')

print(xline_si[0][0])
#400
plt.scatter(xline_si[1][0], yline_si[1][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[1][1], yline_si[1][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[1][2], yline_si[1][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[1][3], yline_si[1][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#600
plt.scatter(xline_si[2][0], yline_si[2][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[2][1], yline_si[2][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[2][2], yline_si[2][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[2][3], yline_si[2][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#800
plt.scatter(xline_si[3][0], yline_si[3][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[3][1], yline_si[3][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[3][2], yline_si[3][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[3][3], yline_si[3][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)
#1000
plt.scatter(xline_si[4][0], yline_si[4][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100)
plt.scatter(xline_si[4][1], yline_si[4][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100)
plt.scatter(xline_si[4][2], yline_si[4][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100)
plt.scatter(xline_si[4][3], yline_si[4][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100)

#Ge
#200
plt.scatter(xline[0][0], yline[0][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-r50')
plt.scatter(xline[0][1], yline[0][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-r125')
plt.scatter(xline[0][2], yline[0][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-r250')
plt.scatter(xline[0][3], yline[0][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-r271')

#400
plt.scatter(xline[1][0], yline[1][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[1][1], yline[1][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[1][2], yline[1][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[1][3], yline[1][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#600
plt.scatter(xline[2][0], yline[2][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[2][1], yline[2][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[2][2], yline[2][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[2][3], yline[2][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#800
plt.scatter(xline[3][0], yline[3][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[3][1], yline[3][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[3][2], yline[3][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[3][3], yline[3][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)
#1000
plt.scatter(xline[4][0], yline[4][0],facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100)
plt.scatter(xline[4][1], yline[4][1],facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100)
plt.scatter(xline[4][2], yline[4][2],facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100)
plt.scatter(xline[4][3], yline[4][3],facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100)

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(400, 68,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(110, 168,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_TempD)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_TempD)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(400, 60), xytext=(400, 60), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(110, 160), xytext=(110, 160), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n Temperature difference (K)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 


plt.show()  